In [1]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, Lasso
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler



# Charger les données
data_train = pd.read_csv('data/data_train.csv', delimiter='\t', quotechar='"')

# Préparer les données pour l'imputation
data_not_null = data_train.dropna(subset=['G1', 'G2'])

# Modèle pour imputer G1 en utilisant G2
model_G1 = LinearRegression()
model_G1.fit(data_not_null[['G2']], data_not_null['G1'])

# Modèle pour imputer G2 en utilisant G1
model_G2 = LinearRegression()
model_G2.fit(data_not_null[['G1']], data_not_null['G2'])

# Imputer G1 pour les valeurs manquantes en utilisant G2
missing_G1 = data_train['G1'].isnull() & data_train['G2'].notnull()
data_train.loc[missing_G1, 'G1'] = model_G1.predict(data_train.loc[missing_G1, ['G2']])

# Imputer G2 pour les valeurs manquantes en utilisant G1
missing_G2 = data_train['G2'].isnull() & data_train['G1'].notnull()
data_train.loc[missing_G2, 'G2'] = model_G2.predict(data_train.loc[missing_G2, ['G1']])

# Supprimer les lignes avec des valeurs manquantes restantes
data_train = data_train.dropna(subset=['G1', 'G2'])



# Imputer les valeurs manquantes restantes avec la moyenne
# data_train['G1'].fillna(data_train['G1'].mean(), inplace=True)
# data_train['G2'].fillna(data_train['G2'].mean(), inplace=True)
data_train_cleaned = data_train

# Convertir les variables catégoriques en variables numériques (encodage one-hot)
data_train_encoded = pd.get_dummies(data_train_cleaned, drop_first=True)

In [2]:
data_train_encoded.columns

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

In [3]:
data_train_encoded["address_U"].value_counts()

address_U
True     279
False    120
Name: count, dtype: int64

In [4]:
display(data_train_encoded.columns)
# Créer une nouvelle colonne pour la consommation totale d'alcool
data_train_encoded['Total_Alcohol_Consumption'] = data_train_encoded['Walc'] + data_train_encoded['Dalc']
data_train_encoded['Combined_G1_G2'] = data_train_encoded['G1'] + data_train_encoded['G2']
data_train_encoded['Combined_Parent_Education'] = data_train_encoded['Medu'] * data_train_encoded['Fedu']
data_train_encoded['Travel_Effect'] = data_train_encoded['address_U'] * data_train_encoded['traveltime']
data_train_encoded['Family_Dynamics'] = data_train_encoded['famsize_LE3'] * data_train_encoded['famrel']
selected_features = [col for col in data_train_encoded.columns if col in ["Combined_G1_G2", 'Combined_Parent_Education', 'Total_Alcohol_Consumption', 'Family_Dynamics','Travel_Effect', 'studytime','failures',"sex_M","school_MS","higher_yes","internet_yes","schoolsup_yes"]]

# Ajouter les nouvelles colonnes encodées manuellement si besoin
X_encoded = data_train_encoded[selected_features]
y_encoded = data_train_encoded['G3']
print(len(X_encoded))
print(len(y_encoded))


Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

399
399


In [5]:
print(X_encoded.shape)
print(y_encoded.shape)


(399, 12)
(399,)


In [6]:
y_encoded.info()

<class 'pandas.core.series.Series'>
Index: 399 entries, 1 to 433
Series name: G3
Non-Null Count  Dtype
--------------  -----
399 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [7]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Définir la grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
}

# Instancier le modèle XGBoost
xgboost_model = xgb.XGBRegressor(random_state=0)

# Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=10, scoring='r2')
grid_search.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres pour XGBoost: {grid_search.best_params_}")

# Utiliser le meilleur modèle
best_xgb_model = grid_search.best_estimator_


Meilleurs hyperparamètres pour XGBoost: {'max_depth': 3, 'n_estimators': 50}


## Test avec cross validation

In [8]:
# Les modèles à tester
from math import sqrt

#GridSearchCV ou RandomizedSearchCV. pour trouver les hyper paramétre
models = {
    'Ridge': Ridge(
        alpha=1.0,                 # Paramètre de régularisation
        solver='auto',             # Méthode de résolution
        max_iter=None,             # Nombre maximal d'itérations
        random_state=None,         # Graîne aléatoire
        tol=1e-3                   # Tolérance pour la convergence
    ),
    
    'Lasso': Lasso(
        alpha=1.0,                 # Paramètre de régularisation
        max_iter=1000,             # Nombre maximal d'itérations
        tol=1e-4,                  # Tolérance pour la convergence
        random_state=None          # Graîne aléatoire
    ),
    
    'XGBoost': best_xgb_model,
    
    'LightGBM': lgb.LGBMRegressor(
        boosting_type='gbdt',      # Type de boosting ('gbdt', 'dart', 'goss')
        learning_rate=0.1,         # Taux d'apprentissage
        n_estimators=100,          # Nombre d'arbres
        max_depth=-1,              # Profondeur maximale des arbres (-1 signifie pas de limite)
        subsample=1.0,             # Proportion des données utilisées pour chaque arbre
        colsample_bytree=1.0,      # Proportion de colonnes échantillonnées par arbre
        random_state=None          # Graîne aléatoire
    ),
    
    'Random Forest': RandomForestRegressor(
        n_estimators=100,          # Nombre d'arbres dans la forêt
        max_depth=None,            # Profondeur maximale des arbres
        min_samples_split=2,       # Nombre minimal d'échantillons pour diviser un nœud
        min_samples_leaf=1,        # Nombre minimal d'échantillons dans une feuille
        random_state=None,         # Graîne aléatoire
        n_jobs=-1                  # Utilisation de tous les cœurs disponibles
    ),
    
    'Linear Regression': LinearRegression(
        fit_intercept=True,        # Estimer une ordonnée à l'origine
        n_jobs=None                # Utilisation de tous les cœurs disponibles
    )
}
# Créer le scorer pour la MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}
# Itérer sur les modèles
for model_name, model in models.items():
    # Validation croisée avec R² comme métrique
    cv_scores_r2 = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring='r2')
    mean_r2 = cv_scores_r2.mean()  # Moyenne des scores R²

    # Validation croisée avec MSE comme métrique
    cv_scores_mse = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring=mse_scorer)
    mean_mse = -cv_scores_mse.mean()  # Moyenne de la MSE
    rmse = sqrt(mean_mse)  # Calcul du RMSE

    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 2.0573
  RMSE: 1.4343
  Mean R²: 0.7702
Lasso:
  Mean MSE: 2.0702
  RMSE: 1.4388
  Mean R²: 0.7790
XGBoost:
  Mean MSE: 2.6329
  RMSE: 1.6226
  Mean R²: 0.7119
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000410 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 93
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 12
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

## Test sans cross validation

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Diviser les données en jeu d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}

# Itérer sur les modèles
for model_name, model in models.items():
    # Entraîner le modèle
    model.fit(X_train, y_train)
    
    # Prédire sur le jeu de test
    y_pred = model.predict(X_test)
    
    # Calculer les métriques
    mean_mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mean_mse)
    mean_r2 = r2_score(y_test, y_pred)
    
    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 0.9548
  RMSE: 0.9771
  Mean R²: 0.8911
Lasso:
  Mean MSE: 0.8804
  RMSE: 0.9383
  Mean R²: 0.8996
XGBoost:
  Mean MSE: 2.7154
  RMSE: 1.6478
  Mean R²: 0.6904
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 86
[LightGBM] [Info] Number of data points in the train set: 319, number of used features: 12
[LightGBM] [Info] Start training from score 11.824451
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

## Deuxième essai

In [10]:
data_train_encoded["address_U"].value_counts()
display(data_train_encoded.columns)
# Créer une nouvelle colonne pour la consommation totale d'alcool
data_train_encoded['Total_Alcohol_Consumption'] = data_train_encoded['Walc'] + data_train_encoded['Dalc']
data_train_encoded['Combined_G1_G2'] = data_train_encoded['G1'] + data_train_encoded['G2']
data_train_encoded['Combined_Parent_Education'] = data_train_encoded['Medu'] * data_train_encoded['Fedu']
data_train_encoded['Travel_Effect'] = data_train_encoded['address_U'] * data_train_encoded['traveltime']
data_train_encoded['Family_Dynamics'] = data_train_encoded['famsize_LE3'] * data_train_encoded['famrel']
selected_features = [col for col in data_train_encoded.columns if
                     col in ["G1", "G2", 'Combined_Parent_Education', 'Total_Alcohol_Consumption',
                             'Family_Dynamics', 'Travel_Effect', 'studytime', 'failures', "sex_M", "school_MS",
                             "higher_yes", "internet_yes", "schoolsup_yes"]]

# Ajouter les nouvelles colonnes encodées manuellement si besoin
X_encoded = data_train_encoded[selected_features]
y_encoded = data_train_encoded['G3']
print(len(X_encoded))
print(len(y_encoded))

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes', 'Total_Alcohol_Consumption', 'Combined_G1_G2',
       'Combined_Parent_Education', 'Travel_Effect', 'Family_Dynamics'],
      dtype='object')

399
399


In [11]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Définir la grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
}

# Instancier le modèle XGBoost
xgboost_model = xgb.XGBRegressor(random_state=0)

# Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=10, scoring='r2')
grid_search.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres pour XGBoost: {grid_search.best_params_}")

# Utiliser le meilleur modèle
best_xgb_model = grid_search.best_estimator_


Meilleurs hyperparamètres pour XGBoost: {'max_depth': 6, 'n_estimators': 50}


In [12]:
# Les modèles à tester
from math import sqrt

#GridSearchCV ou RandomizedSearchCV. pour trouver les hyper paramétre
models = {
    'Ridge': Ridge(
        alpha=1.0,                 # Paramètre de régularisation
        solver='auto',             # Méthode de résolution
        max_iter=None,             # Nombre maximal d'itérations
        random_state=None,         # Graîne aléatoire
        tol=1e-3                   # Tolérance pour la convergence
    ),

    'Lasso': Lasso(
        alpha=1.0,                 # Paramètre de régularisation
        max_iter=1000,             # Nombre maximal d'itérations
        tol=1e-4,                  # Tolérance pour la convergence
        random_state=None          # Graîne aléatoire
    ),

    'XGBoost': best_xgb_model,

    'LightGBM': lgb.LGBMRegressor(
        boosting_type='gbdt',      # Type de boosting ('gbdt', 'dart', 'goss')
        learning_rate=0.1,         # Taux d'apprentissage
        n_estimators=100,          # Nombre d'arbres
        max_depth=-1,              # Profondeur maximale des arbres (-1 signifie pas de limite)
        subsample=1.0,             # Proportion des données utilisées pour chaque arbre
        colsample_bytree=1.0,      # Proportion de colonnes échantillonnées par arbre
        random_state=None          # Graîne aléatoire
    ),

    'Random Forest': RandomForestRegressor(
        n_estimators=100,          # Nombre d'arbres dans la forêt
        max_depth=None,            # Profondeur maximale des arbres
        min_samples_split=2,       # Nombre minimal d'échantillons pour diviser un nœud
        min_samples_leaf=1,        # Nombre minimal d'échantillons dans une feuille
        random_state=None,         # Graîne aléatoire
        n_jobs=-1                  # Utilisation de tous les cœurs disponibles
    ),

    'Linear Regression': LinearRegression(
        fit_intercept=True,        # Estimer une ordonnée à l'origine
        n_jobs=None                # Utilisation de tous les cœurs disponibles
    )
}
# Créer le scorer pour la MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}
# Itérer sur les modèles
for model_name, model in models.items():
    # Validation croisée avec R² comme métrique
    cv_scores_r2 = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring='r2')
    mean_r2 = cv_scores_r2.mean()  # Moyenne des scores R²

    # Validation croisée avec MSE comme métrique
    cv_scores_mse = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring=mse_scorer)
    mean_mse = -cv_scores_mse.mean()  # Moyenne de la MSE
    rmse = sqrt(mean_mse)  # Calcul du RMSE

    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 1.8770
  RMSE: 1.3700
  Mean R²: 0.7894
Lasso:
  Mean MSE: 2.0305
  RMSE: 1.4249
  Mean R²: 0.7865
XGBoost:
  Mean MSE: 2.5818
  RMSE: 1.6068
  Mean R²: 0.7224
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000607 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 98
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 13
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with

## Troisième essai avec ajout des test des hyperparamètres pour chaque modèle

In [127]:
data_train_encoded["address_U"].value_counts()
display(data_train_encoded.columns)
# Créer une nouvelle colonne pour la consommation totale d'alcool
data_train_encoded['Total_Alcohol_Consumption'] = data_train_encoded['Walc'] + data_train_encoded['Dalc']
data_train_encoded['Combined_G1_G2'] = data_train_encoded['G1'] + data_train_encoded['G2']
data_train_encoded['Combined_Parent_Education'] = data_train_encoded['Medu'] * data_train_encoded['Fedu']
data_train_encoded['Travel_Effect'] = data_train_encoded['address_U'] * data_train_encoded['traveltime']
data_train_encoded['Family_Dynamics'] = data_train_encoded['famsize_LE3'] * data_train_encoded['famrel']
selected_features = [col for col in data_train_encoded.columns if
                     col in ["G1", "G2", 'Combined_Parent_Education', 'Total_Alcohol_Consumption', 'Family_Dynamics', 'Travel_Effect', 'studytime', 'failures',"schoolsup_yes"]]

# ,"internet_yes","schoolsup_yes"
# Ajouter les nouvelles colonnes encodées manuellement si besoin
X_encoded = data_train_encoded[selected_features]
y_encoded = data_train_encoded['G3']
print(len(X_encoded))
print(len(y_encoded))

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes', 'Total_Alcohol_Consumption', 'Combined_G1_G2',
       'Combined_Parent_Education', 'Travel_Effect', 'Family_Dynamics',
       'Study_Failures_Interaction', 'Parent_Education',
       'Absences_Health_Interaction', 'Study_Activities_Interaction',
       'Health_Alcohol_Interaction'],
      dtype='object')

399
399


In [128]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Définir la grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
}

# Instancier le modèle XGBoost
xgboost_model = xgb.XGBRegressor(random_state=0)

# Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid, cv=10, scoring='r2')
grid_search.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres
print(f"Meilleurs hyperparamètres pour XGBoost: {grid_search.best_params_}")

# Utiliser le meilleur modèle
best_xgb_model = grid_search.best_estimator_


Meilleurs hyperparamètres pour XGBoost: {'max_depth': 3, 'n_estimators': 50}


In [132]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Grille d'hyperparamètres pour Ridge
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sag']
}

# Instancier le modèle
ridge_model = Ridge()

# Utiliser GridSearchCV
grid_search_ridge = GridSearchCV(estimator=ridge_model, param_grid=param_grid_ridge, cv=10, scoring='r2')
grid_search_ridge.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres pour Ridge
print(f"Meilleurs hyperparamètres pour Ridge: {grid_search_ridge.best_params_}")

# Utiliser le meilleur modèle
best_ridge_model = grid_search_ridge.best_estimator_


Meilleurs hyperparamètres pour Ridge: {'alpha': 10.0, 'solver': 'lsqr'}


In [133]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Grille d'hyperparamètres pour RandomForest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Instancier le modèle
rf_model = RandomForestRegressor(random_state=0)

# Utiliser GridSearchCV
grid_search_rf = GridSearchCV(estimator=rf_model, param_grid=param_grid_rf, cv=10, scoring='r2')
grid_search_rf.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres pour RandomForest
print(f"Meilleurs hyperparamètres pour RandomForest: {grid_search_rf.best_params_}")

# Utiliser le meilleur modèle
best_rf_model = grid_search_rf.best_estimator_


Meilleurs hyperparamètres pour RandomForest: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 50}


In [134]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

# Grille d'hyperparamètres pour LightGBM
param_grid_lgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Instancier le modèle
lgb_model = lgb.LGBMRegressor()

# Utiliser RandomizedSearchCV
random_search_lgb = RandomizedSearchCV(estimator=lgb_model, param_distributions=param_grid_lgb, n_iter=10, cv=10, scoring='r2', random_state=0)
random_search_lgb.fit(X_encoded, y_encoded)

# Afficher les meilleurs hyperparamètres pour LightGBM
print(f"Meilleurs hyperparamètres pour LightGBM: {random_search_lgb.best_params_}")

# Utiliser le meilleur modèle
best_lgb_model = random_search_lgb.best_estimator_


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000363 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 9
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [135]:
# Les modèles à tester
from math import sqrt

#GridSearchCV ou RandomizedSearchCV. pour trouver les hyper paramétre
models = {
    'Ridge': best_ridge_model,

    'Lasso': Lasso(
        alpha=1.0,                 # Paramètre de régularisation
        max_iter=1000,             # Nombre maximal d'itérations
        tol=1e-4,                  # Tolérance pour la convergence
        random_state=None          # Graîne aléatoire
    ),

    'XGBoost': best_xgb_model,

    'LightGBM': best_lgb_model,

    'Random Forest': best_rf_model,

    'Linear Regression': LinearRegression(
        fit_intercept=True,        # Estimer une ordonnée à l'origine
        n_jobs=None                # Utilisation de tous les cœurs disponibles
    )
}
# Créer le scorer pour la MSE
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
# Initialisation pour suivre le meilleur modèle basé sur le R²
best_model_name = None
best_r2_score = float('-inf')  # Commence avec une valeur très basse
best_model_results = {}
# Itérer sur les modèles
for model_name, model in models.items():
    # Validation croisée avec R² comme métrique
    cv_scores_r2 = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring='r2')
    mean_r2 = cv_scores_r2.mean()  # Moyenne des scores R²

    # Validation croisée avec MSE comme métrique
    cv_scores_mse = cross_val_score(model, X_encoded, y_encoded, cv=10, scoring=mse_scorer)
    mean_mse = -cv_scores_mse.mean()  # Moyenne de la MSE
    rmse = sqrt(mean_mse)  # Calcul du RMSE

    # Affichage des résultats pour chaque modèle
    print(f'{model_name}:')
    print(f'  Mean MSE: {mean_mse:.4f}')
    print(f'  RMSE: {rmse:.4f}')
    print(f'  Mean R²: {mean_r2:.4f}')

    # Vérifier si ce modèle a le meilleur R²
    if mean_r2 > best_r2_score:
        best_r2_score = mean_r2
        best_model_name = model_name
        best_model = model.fit(X_encoded, y_encoded)
        best_model_results = {
            'Mean MSE': mean_mse,
            'RMSE': rmse,
            'R²': mean_r2
        }

# Afficher le meilleur modèle basé sur le R²
print(f'Best Model: {best_model_name}')
print(f'Mean MSE: {best_model_results["Mean MSE"]:.4f}')
print(f'RMSE: {best_model_results["RMSE"]:.4f}')
print(f'R2 Score: {best_model_results["R²"]:.4f}')


Ridge:
  Mean MSE: 1.8421
  RMSE: 1.3572
  Mean R²: 0.7962
Lasso:
  Mean MSE: 2.0305
  RMSE: 1.4249
  Mean R²: 0.7865
XGBoost:
  Mean MSE: 2.6740
  RMSE: 1.6352
  Mean R²: 0.7156
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 359, number of used features: 9
[LightGBM] [Info] Start training from score 11.891365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

## Prédiction

In [136]:
data_test = pd.read_csv('data/data_test.csv', delimiter='\t', quotechar='"')

display(data_test)

# Encodage des variables catégoriques dans data_test en utilisant le même encodage que pour data_train_cleaned
X_test_encoded = pd.get_dummies(data_test, drop_first=True)

# Assurer que les colonnes dans X_test_encoded correspondent à celles dans X_encoded (qui a été utilisé pour entraîner le modèle)
missing_cols = set(X_encoded.columns) - set(X_test_encoded.columns)
for col in missing_cols:
    X_test_encoded[col] = 0

# Réordonner les colonnes pour correspondre à l'ordre de X_encoded
X_test_encoded = X_test_encoded[X_encoded.columns]

# Faire des prédictions sur data_test
y_test_pred = best_model.predict(X_test_encoded)

y_test_pred

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
434,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,4,3,4,1,1,3,4,0,11
435,GP,F,17,U,GT3,T,1,1,at_home,other,...,no,5,3,3,1,1,3,2,9,11
436,GP,F,15,U,LE3,T,1,1,at_home,other,...,no,4,3,2,2,3,3,6,12,13
437,GP,M,15,U,LE3,A,3,2,services,other,...,no,4,2,2,1,1,1,0,15,16
438,GP,M,15,U,GT3,A,2,2,other,other,...,yes,4,5,2,1,1,3,0,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,MS,M,19,R,GT3,T,1,1,other,services,...,no,4,3,2,1,3,5,0,5,8
646,MS,F,18,R,GT3,T,2,2,at_home,other,...,no,5,3,3,1,3,4,0,14,17
647,MS,F,18,R,GT3,T,4,4,teacher,at_home,...,yes,4,4,3,2,2,5,4,7,9
648,MS,F,18,U,LE3,T,3,1,teacher,services,...,no,4,3,4,1,1,1,4,15,15


array([ 9.75009819, 11.27678473, 13.06268601, 16.46581226, 14.65607909,
       13.64404394, 14.07472116, 12.33218624, 11.44900283, 12.46103799,
       11.32015108, 15.4537771 ,  9.94109514, 11.03861569,  9.55378427,
       13.42970679, 15.58262885, 15.7114806 , 16.72351576, 10.06994689,
       12.30835435,  9.44321968, 11.96316252, 11.83431077,  8.44934766,
        9.35773428, 11.66333998, 15.66811425, 13.6019249 , 12.58864243,
       10.82427855, 12.67537514, 13.3442214 ,  9.05791174, 16.68014941,
       10.95112737, 12.80422689, 13.3442214 ,  9.05791174, 11.49112188,
        9.03070559, 10.56581943, 11.50385164, 14.52722734, 13.68741029,
       11.74882538, 10.11674369,  9.09322608, 11.08073474, 12.54652339,
        9.42493252,  8.00251023,  9.95725535, 10.31959838, 10.39360132,
       17.60669917, 12.54652339,  9.76812142,  9.76812142, 11.53448823,
        9.55378427, 11.66333998, 10.56581943, 15.16543701, 10.65130482,
       12.05065085, 11.53448823, 11.66333998,  9.76812142, 13.34

In [137]:
predictions_valid_rounded = [round(prediction) for prediction in y_test_pred]
print(y_test_pred)

[ 9.75009819 11.27678473 13.06268601 16.46581226 14.65607909 13.64404394
 14.07472116 12.33218624 11.44900283 12.46103799 11.32015108 15.4537771
  9.94109514 11.03861569  9.55378427 13.42970679 15.58262885 15.7114806
 16.72351576 10.06994689 12.30835435  9.44321968 11.96316252 11.83431077
  8.44934766  9.35773428 11.66333998 15.66811425 13.6019249  12.58864243
 10.82427855 12.67537514 13.3442214   9.05791174 16.68014941 10.95112737
 12.80422689 13.3442214   9.05791174 11.49112188  9.03070559 10.56581943
 11.50385164 14.52722734 13.68741029 11.74882538 10.11674369  9.09322608
 11.08073474 12.54652339  9.42493252  8.00251023  9.95725535 10.31959838
 10.39360132 17.60669917 12.54652339  9.76812142  9.76812142 11.53448823
  9.55378427 11.66333998 10.56581943 15.16543701 10.65130482 12.05065085
 11.53448823 11.66333998  9.76812142 13.3442214  16.55129766 11.40563648
 13.42970679 13.19153776  9.51041792 10.31959838  8.42437982  7.32685927
 10.73679022  9.22207783  7.61519936 14.78493084 15.5

In [138]:
df = pd.DataFrame(y_test_pred)
df.to_csv('my_pred.csv', index=False)